## Runnables

There are many Runnable classes for simplifying the tasks in Langchain chains  
 - **RunnablePassthrogh**: Return Input as output as is
 - **RunnbaleLambda**: Run a function and return result of function. Lambda functions can be used here.
 - **RunnableParallel**: Run multiple chains/runnables in parallel

#### Lets Create 2 LLMs which we will use for testing

In [17]:
from langchain_ollama.chat_models import ChatOllama

llm1 = ChatOllama(
    base_url = 'http://localhost:11434',
    model = 'qwen2.5:0.5b'
)

llm2 = ChatOllama(
    base_url = 'http://localhost:11434',
    model = 'llama3.2:1b'
)

#### Runnable Passthrough

In [9]:
from langchain_core.runnables import RunnablePassthrough
r1 = RunnablePassthrough()
print(r1.invoke("Hello"))

Hello


#### Runnable Lambda

In [10]:
from langchain_core.runnables import RunnableLambda

# Create some function
def some_func(input: str) -> str:
    return f"Hello {input}"

r2 = RunnableLambda(some_func)
print(r2.invoke("Sam"))

Hello Sam


#### Runnable Parallel

In [12]:
from langchain_core.runnables import RunnableParallel

r3 = RunnableParallel(out1=r1, out2=r2)
print(r3.invoke("John"))

{'out1': 'John', 'out2': 'Hello John'}


#### Lets try to make some useful app from these learned runnables
We will give 1 topic to 2 LLMs and ask them to explain the topic to a 10 year old child.  
And then We will, evaluate which explanation is better using one of the LLM model

#### Prompt

In [14]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template="""
    User will give a topic, your task is to explain the topic to a 10 year old child. 
    The explanation must not contain complex sentence structure as it may confuse the kid. 
    Generate explanation in simple words with few examples to explain the topic clearly. 

    Topic: {topic}
    """,
    input_variables = ['topic']
)

#### First Chain from 1st LLM to generate explanation

In [18]:
from langchain_core.output_parsers import StrOutputParser
ch1 = prompt | llm1 | StrOutputParser()

# Test
ch1.invoke({"topic": "Gravity"})

"Gravity is like a strong invisible force that pulls objects towards the Earth, similar to how you can pull on your friend's backpack or toy cars and they fall down. It’s what makes the sky, sun, and moon stay up in our world. We can lift things with gravity too – like when you put toys under the table to keep them from falling off. But remember, if you drop an object, it will hurt! Gravity is super important for us because without it, we couldn't make tall buildings or use the stars as a guide for travel. Isn’t that neat?"

#### Second chain from other LLM model to generate explanation

In [21]:
ch2 = prompt | llm2 | StrOutputParser()

# Test
ch2.invoke({"topic": "Gravity"})

"Hey there, kiddo! Let's talk about gravity.\n\nGravity is like a strong hug from the Earth. It's what keeps you on the ground and what makes things fall down instead of flying up into space.\n\nImagine you're playing with a ball. You throw it up in the air, right? What happens to it? It comes back down to the ground, right?\n\nThat's because the Earth is giving the ball a big hug, pulling it towards itself. This is called gravity. The more massive something is, like the Earth or a planet, the stronger its gravity is.\n\nBut why does that happen? Well, imagine you're on a merry-go-round. If you spin really fast, you'll want to keep going in a circle because it's hard to stop spinning. That's kind of like what's happening with gravity and objects near the Earth.\n\nThe Earth is so massive that its gravity is pulling everything towards it. This includes not just planets but also stars, even other galaxies! So when an object falls down from space, it comes back down to the Earth because w

#### So both the Explanations work, lets try to evaluate which explanation if better

In [22]:
eval_prompt = PromptTemplate(
    template="""
    You are an evaluator who will evaluate the 2 explanations provided by 2 LLM models and tell which explanation is better and why. 
    There is a topic for which 2 LLM models have generated an explanation for 10 year old kid. Evaluate both the explanations and 
    in response tell which explanation is better suitable for 10 year old kid and why.
    -------------------------------------------------
    Topic: {topic}
    -------------------------------------------------
    Explanation 1: {explanation_1}
    -------------------------------------------------
    Explanation 2: {explanation_2}
    -------------------------------------------------
    Evaluation:
    """,
    input_variables = ['topic', 'explanation_1', 'explanation_2']
)

In [26]:
eval_chain = RunnableParallel(explanation_1=ch1, explanation_2=ch2, topic=RunnablePassthrough()) | eval_prompt | llm2 | StrOutputParser()
ev = eval_chain.invoke({"topic": "Gravity"})
print(ev)

I'll evaluate both explanations provided by two LLM models and identify which one is better suitable for a 10-year-old kid.

**Explanation 1**

This explanation uses simple and relatable analogies to explain gravity, such as comparing it to a "big hug from the Earth." It also uses examples that are easy to understand, like rolling a ball down a hill or throwing a toy car up in the air. The language is clear and concise, making it accessible to 10-year-olds.

However, there are some minor issues with this explanation:

* It assumes that the child has already learned about gravity before reading this explanation.
* It uses a bit of a simplification by saying "we can't move the ball by ourselves" - in real life, people can still move objects even if they have to work hard.

**Explanation 2**

This explanation also tries to use relatable analogies to explain gravity. However, it takes some creative liberties with the analogy, such as comparing gravity to a marble rolling down a hill. This 

#### The Evaluation is Good, But can't see what explanation was geenrated by Each LLM Model.
In order provide send the Output of Models, we can use RunParallel, Send output of LLMs as passthrough in Parallel with Evaluations

In [29]:
eval_chain_logic = eval_prompt | llm2 | StrOutputParser()

eval_chain2 = RunnableParallel(explanation_1=ch1, explanation_2=ch2, topic=RunnablePassthrough()) | \
                RunnableParallel(evaluation=eval_chain_logic, inputs=RunnablePassthrough())

ev2 = eval_chain2.invoke({"topic": "Gravity"})

print("Explanation 1: ", ev2["inputs"]["explanation_1"], end="\n\n")
print("Explanation 2: ", ev2["inputs"]["explanation_2"], end="\n\n")
print("Final Evaluation: ", ev2["evaluation"])

Explanation 1:  Hey there, kiddo! Let's talk about something cool called gravity.

You know how things fall down when you drop them? Like a ball or a toy? That's because of something called gravity. It's like an invisible force that pulls everything towards each other!

Imagine you have two friends, one on the playground and one on another part of the playground. If they're close to each other, they'll stick together, right? That's kind of like what gravity does with things on Earth.

When something is near the ground, it gets pulled down by this invisible force. It's like the Earth is saying, "Hey, I want you to come closer!" And because of that, things don't just float away into space.

For example, if you drop a toy on the ground, it will fall down towards the Earth. That's gravity doing its thing! And we're all stuck to the ground because of gravity too.

So, gravity is like a big hug from the Earth, keeping everything in place. It's really cool and helps us live on this amazing pl